Calcolo del grafo a partire dal dataset iniziale. Fornisce il dataset di input per il Main()

In [183]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
pd.set_option('display.max_colwidth', 4000)

In [2]:
df = pd.read_excel('C:\\Users\michele.lenato\Downloads\worldcities.xlsx', sheet_name='Sheet1')
df['population'].fillna(0, inplace=True)
#ordinamento longitudine
cols = ['lng']  #
XX = df[['city', 'lat', 'lng', 'iso3', 'population', 'id']]  #dataset su cui si sono fatte le query sottostanti
XX = XX.sort_values(by=cols, ascending=True)
ll = [(l1, l2) for (l1, l2) in zip(XX.loc[XX.index]['lat'], XX.loc[XX.index]['lng'])]
XX['ll']=ll
GDM = pd.DataFrame()
a_est = XX.loc[34:]  # 34 indice di Londra
a_ovest = XX.loc[:34]
GDM = pd.concat([a_est, a_ovest]).drop_duplicates()
ll = [(l1, l2) for (l1, l2) in zip(GDM.loc[GDM.index]['lat'], GDM.loc[GDM.index]['lng'])]
GDM['ll']=ll
#ordinamento latitudine
cols2 = ['lat']  
GDM_lat = GDM.sort_values(by=cols2, ascending=True)  #dataset su cui si sono fatte le query sottostanti

In [3]:
X = pd.concat([XX, XX[:150]])
X = X.reset_index()
del X['index']

In [4]:
#id con le 400 latitudini più vicine
diz_lat = {}
n_lat = 400
for i, idx in enumerate(GDM_lat.index):
   l = list(GDM_lat.loc[idx:]['id'][:n_lat]) +list(GDM_lat.loc[:idx]['id'][-n_lat:])  
   diz_lat[GDM_lat.loc[idx]['id']]= l

In [5]:
#id con le 150 longitudini più vicine
diz_lnt = {}
n_lon = 150
for i, idx in enumerate(X.index):
    diz_lnt[X.loc[idx]['id']]=list(X.loc[idx:]['id'][:n_lon])   

Ottengo un dizionario con 
chiave: il nodo A, 
valore: i suoi candidati nodi Z

In [6]:
#Intersezione: id con le longitudini e latitudini più vicine
cluster_ll = {}
saltati = []
for k, v in diz_lnt.items():
  try:
   cluster_ll[k] = [x for x in v if x in diz_lat[k]]
  except KeyError:
    saltati.append(k)
    continue

Lo inserisco in un dataframe

In [7]:
cl = pd.DataFrame(cluster_ll.items(), columns=['nodo_a', 'nodi_z'])
z  = pd.DataFrame(cl['nodi_z'].tolist()) # ogni colonna è un id
z = z.fillna(0)
z.astype(int)
zz = pd.DataFrame()
zz = XX.merge(z, left_on='id', right_on=0) # inner join con il dataset iniziale per aggiungere coordinate, nomi, iso3 e popolazione


In [10]:
zz[:1]

,city,lat,lng,iso3,population,id,ll,0,1,2,...,87,88,89,90,91,92,93,94,95,96
0,Zvëzdnyy,70.9566,-179.59,RUS,10.0,1643877771,"(70.9566, -179.59)",1643877771,1.643467e+09,1.643868e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Aggiunta coordinate dei nodi Z

In [8]:
coord = pd.DataFrame()
for idx in (zz.index):
  temp = pd.DataFrame()
  lista_ll = []
  for x in zz.loc[idx][8:].values.tolist():
    try:
      if x is None or int(x)==0:
        pass
      elif int(x)!=0:
        lista_ll.append(XX[XX['id']==int(x)]['ll'].values[0]) 
    except ValueError:
      pass
  temp['id'] = [int(zz.loc[idx]['id'])]
  temp['llz'] = [lista_ll]
  coord = pd.concat([coord, temp])

fin = pd.DataFrame()
fin = zz.merge(coord, left_on='id', right_on='id')


In [ ]:
fin[:1]

Calcolo distanza tra nodi A e i loro nodi Z

In [9]:
import geopy.distance

def nodi(lista):
  candidati = []
  coord = []
  for valore in lista:
    if type(valore)!=list and valore!=0:
      if type(int(valore))==int :
        candidati.append(int(valore))
      if type(valore)==tuple:
        coord.append(valore)

  return candidati, coord

def primo(lista):
  return lista[0]


def nodi_candidati(fin):
  '''calcola il df "finale" con i nodi A-Z e distanze min, med, max dato il df temporaneo con i possibili candidati'''
  finale = pd.DataFrame()

  for j in range(0,fin.shape[0]): 

    temp = pd.DataFrame()
    nodes = [fin.loc[j,:]['id']] + nodi(fin.loc[j][8:].tolist())[0] #somma primo nodo (A) della lista agli altri (Z)
    coord = [fin.loc[j,:]['ll']] + fin.loc[j]['llz']  #le loro coordinate
    #print(coord)
    cand = []
    cand_fin = []
    for_df = []
    for n, c in zip(nodes, coord):
      cand.append((int(n), c))
   
    for i in range(1, len(cand)):
      dist_a_z = round(geopy.distance.distance(cand[0][1], cand[i][1]).km, 1)
      if dist_a_z > 1.0:  #escludo eventuali nodi A-Z coincidenti
        if (cand[i][0], dist_a_z) not in cand_fin:  #VERIFICO SE NON CI SONO DOPPIONI NELLA LISTA CAND
          cand_fin.append((cand[i][0], dist_a_z))
    for_df = sorted(cand_fin, key=lambda x: x[1])[:3] #prendo i primi 3 Z dopo averli ordinati
    #for_df = sorted(cand_fin, key=lambda x: x[1])[:6] #prendo i primi 6 Z dopo averli ordinati

    if len(for_df)==3:
      temp['nodoA'] = [fin.loc[j,:]['id']] 
      temp['nodoZmin'] = [for_df[0][0]]
      temp['dist_min'] = [for_df[0][1]]
      temp['nodoZmed'] = [for_df[1][0]]
      temp['dist_med'] = [for_df[1][1]]
      temp['nodoZmax'] = [for_df[2][0]]
      temp['dist_max'] = [for_df[2][1]]
      finale = pd.concat([finale, temp])
     
    elif len(for_df)==2:
      temp['nodoA'] = [fin.loc[j,:]['id']]  
      temp['nodoZmin'] = [for_df[0][0]]
      temp['dist_min'] = [for_df[0][1]]
      temp['nodoZmed'] = [for_df[1][0]]
      temp['dist_med'] = [for_df[1][1]]
      finale = pd.concat([finale, temp])
    elif len(for_df)==1:
      temp['nodoA'] = [fin.loc[j,:]['id']]
      temp['nodoZmin'] = [for_df[0][0]]
      temp['dist_min'] = [for_df[0][1]]
      finale = pd.concat([finale, temp])  

    # le seguenti righe sono state usate per il calcolo del grafo finale
    
    elif len(for_df)==4:
      temp['nodoA'] = [fin.loc[j,:]['id']] 
      temp['nodoZmin'] = [for_df[0][0]]
      temp['dist_min'] = [for_df[0][1]]
      temp['nodoZmed'] = [for_df[1][0]]
      temp['dist_med'] = [for_df[1][1]]
      temp['nodoZmax'] = [for_df[2][0]]
      temp['dist_max'] = [for_df[2][1]]
      temp['nodoZmax2'] = [for_df[3][0]]
      temp['dist_max2'] = [for_df[3][1]]
      finale = pd.concat([finale, temp])

    elif len(for_df)==5:
      temp['nodoA'] = [fin.loc[j,:]['id']] 
      temp['nodoZmin'] = [for_df[0][0]]
      temp['dist_min'] = [for_df[0][1]]
      temp['nodoZmed'] = [for_df[1][0]]
      temp['dist_med'] = [for_df[1][1]]
      temp['nodoZmax'] = [for_df[2][0]]
      temp['dist_max'] = [for_df[2][1]]
      temp['nodoZmax2'] = [for_df[3][0]]
      temp['dist_max2'] = [for_df[3][1]]
      temp['nodoZmax3'] = [for_df[4][0]]
      temp['dist_max3'] = [for_df[4][1]]
      finale = pd.concat([finale, temp])


    elif len(for_df)==6:
      temp['nodoA'] = [fin.loc[j,:]['id']] 
      temp['nodoZmin'] = [for_df[0][0]]
      temp['dist_min'] = [for_df[0][1]]
      temp['nodoZmed'] = [for_df[1][0]]
      temp['dist_med'] = [for_df[1][1]]
      temp['nodoZmax'] = [for_df[2][0]]
      temp['dist_max'] = [for_df[2][1]]
      temp['nodoZmax2'] = [for_df[3][0]]
      temp['dist_max2'] = [for_df[3][1]]
      temp['nodoZmax3'] = [for_df[4][0]]
      temp['dist_max3'] = [for_df[4][1]]
      temp['nodoZmax4'] = [for_df[5][0]]
      temp['dist_max4'] = [for_df[5][1]]
      finale = pd.concat([finale, temp])

    elif len(for_df)==0:
      pass
 

  return finale

Dataframe con le distanze dei nodi Z più vicini a ciascun nodo A

In [10]:
datafr = nodi_candidati(fin)

Aggiunta dei campi ISO3 e popolazione dei nodi A e Z

In [11]:
#query su XX per aggiungere iso3 e popolazione, nel caso dei 3 nodi più vicini


#isoA
fin7 = pd.DataFrame()
for n in datafr['nodoA'].values:
  tem = pd.DataFrame()
  try:
      tem['isoA'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoA'] = ['null']
  fin7 = pd.concat([fin7, tem])
  
#isoz1
fin1 = pd.DataFrame()
for n in datafr['nodoZmin'].values:
  tem = pd.DataFrame()
  try:
      tem['isoZ1'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoZ1'] = ['null']
  fin1 = pd.concat([fin1, tem])

#isoz2
fin2 = pd.DataFrame()
for n in datafr['nodoZmed'].values:
  tem = pd.DataFrame()
  try:
      tem['isoZ2'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoZ2'] = ['null']
  fin2 = pd.concat([fin2, tem])

#isoz3
fin3 = pd.DataFrame()
for n in datafr['nodoZmax'].values:
  tem = pd.DataFrame()
  try:
      tem['isoZ3'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoZ3'] = ['null']
  fin3 = pd.concat([fin3, tem])

#pop1
fin4 = pd.DataFrame()
for n in datafr['nodoZmin'].values:
  tem = pd.DataFrame()
  try:
      tem['popZ1'] = [XX[XX['id']==n]['population'].values[0]]
  except IndexError :
      tem['popZ1'] = ['null']
  fin4 = pd.concat([fin4, tem])

#pop2
fin5 = pd.DataFrame()
for n in datafr['nodoZmed'].values:
  tem = pd.DataFrame()
  try:
      tem['popZ2'] = [XX[XX['id']==n]['population'].values[0]]
  except IndexError :
      tem['popZ2'] = ['null']
  fin5 = pd.concat([fin5, tem])

#pop3
fin6 = pd.DataFrame()
for n in datafr['nodoZmax'].values:
  tem = pd.DataFrame()
  try:
      tem['popZ3'] = [XX[XX['id']==n]['population'].values[0]]
  except IndexError :
      tem['popZ3'] = ['null']
  fin6 = pd.concat([fin6, tem])

Aggiunta dei weight in base alle condizioni del progetto su ISO3 e popolazione

In [12]:
def flatten(L):
    for l in L:
        if isinstance(l, list):
            yield from flatten(l)
        else:
            yield l

isoA = list(flatten(list(flatten(fin7.values.tolist()))))
popZ3 = list(flatten(list(flatten(fin6.values.tolist()))))
popZ2 = list(flatten(list(flatten(fin5.values.tolist()))))
popZ1 = list(flatten(list(flatten(fin4.values.tolist()))))
isoZ3 = list(flatten(list(flatten(fin3.values.tolist()))))
isoZ2 = list(flatten(list(flatten(fin2.values.tolist()))))
isoZ1 = list(flatten(list(flatten(fin1.values.tolist()))))

datafr['isoA'] = isoA
datafr['isoZ1'] = isoZ1
datafr['isoZ2'] = isoZ2
datafr['isoZ3'] = isoZ3
datafr['popZ1'] = popZ1
datafr['popZ2'] = popZ2
datafr['popZ3'] = popZ3

In [13]:
def aggiorna_peso_nodo_vicino_b(isoa, iso1z,pop1z):
    '''calcola il peso minimo secondo le specifiche per il nodo vicino'''
 
    if iso1z!='null':
      if isoa != iso1z and pop1z>200000:
        return 6
      elif isoa != iso1z and pop1z<=200000:
        return 4
      elif isoa == iso1z and pop1z>200000:
        return 4
      elif isoa == iso1z and pop1z<=200000:
        return 2
      elif isoa != iso1z and pop1z==0:
        return 4
      elif isoa == iso1z and pop1z==0:
        return 2

    elif iso1z == 0:
      return 9999999   
    elif iso1z == 'null':
      return 9999999   


def aggiorna_peso_nodo_medio_b(isoa, iso2z,pop2z):
    '''calcola il peso medio secondo le specifiche per il nodo vicino'''
    if iso2z!='null' :
      if isoa != iso2z and pop2z>200000:
        return 8
      elif isoa != iso2z and pop2z<=200000:
        return 6
      elif isoa != iso2z and pop2z<=200000:
        return 6
      elif isoa == iso2z and pop2z>200000:
        return 6
      elif isoa == iso2z and pop2z<=200000:
        return 4
      elif isoa != iso2z and pop2z==0:
        return 6
      elif isoa == iso2z and pop2z==0:
        return 4
    elif iso2z == 0:
      return 9999999   
    elif iso2z == 'null':
      return 9999999   


def aggiorna_peso_nodo_massimo_b(isoa, iso3z,pop3z):
    '''calcola il peso massimo secondo le specifiche per il nodo vicino'''
    if iso3z!='null' :
      if isoa != iso3z and pop3z>200000:
        return 12
      elif isoa != iso3z and pop3z<=200000:
        return 10
      elif isoa == iso3z and pop3z>200000:
        return 10
      elif isoa == iso3z and pop3z<=200000:
        return 8
      elif isoa != iso3z and pop3z==0:
        return 10
      elif isoa == iso3z and pop3z==0:
        return 8

    elif iso3z == 0:
      return 9999999   
    elif iso3z == 'null':
      return 9999999   

In [14]:
datafr['weight_nodo_vicino'] = [aggiorna_peso_nodo_vicino_b( isoA, isoZ1,popZ1) for isoA, isoZ1,popZ1 in zip( datafr.isoA.values, datafr.isoZ1.values, datafr.popZ1.values)]
datafr['weight_nodo_medio'] = [aggiorna_peso_nodo_medio_b(isoA,  isoZ2, popZ2) for  isoA, isoZ2,popZ2 in zip(  datafr.isoA.values, datafr.isoZ2.values, datafr.popZ2.values)]
datafr['weight_nodo_massimo'] = [aggiorna_peso_nodo_massimo_b( isoA, isoZ3,popZ3) for isoA, isoZ3,popZ3 in zip( datafr.isoA.values, datafr.isoZ3.values, datafr.popZ3.values)]

In [15]:
datafr

,nodoA,nodoZmin,dist_min,nodoZmed,dist_med,nodoZmax,dist_max,isoA,isoZ1,isoZ2,isoZ3,popZ1,popZ2,popZ3,weight_nodo_vicino,weight_nodo_medio,weight_nodo_massimo
0,1643877771,1643466779,517.2,1.643693e+09,544.0,1.643387e+09,666.4,RUS,RUS,RUS,RUS,2248.0,297,776,2,4,8
0,1643466779,1643693470,333.4,1.643868e+09,348.5,1.643889e+09,379.9,RUS,RUS,RUS,RUS,297.0,2536,1242,2,4,8
0,1876731744,1882504626,602.7,1.882879e+09,629.2,1.882940e+09,632.6,WLF,WSM,WSM,WSM,0.0,0,0,4,6,10
0,1776737461,1776829745,303.5,1.184218e+09,1602.6,1.258907e+09,2720.7,TON,TON,COK,PYF,7391.0,5445,131695,2,6,10
0,1776829745,1882940042,568.7,1.882048e+09,570.5,1.882030e+09,572.3,TON,WSM,WSM,WSM,0.0,0,0,4,6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1242615095,1242640119,16.9,1.242741e+09,212.7,1.876732e+09,559.7,FJI,FJI,FJI,WLF,47604.0,27949,0,2,4,10
0,1242640119,1242740801,196.6,1.876732e+09,542.9,1.776737e+09,740.2,FJI,FJI,WLF,TON,27949.0,0,24571,2,6,10
0,1798251391,1876731744,700.2,1.242741e+09,875.9,1.882505e+09,1047.9,TUV,WLF,FJI,WSM,0.0,27949,0,4,6,10
0,1643017874,1643466779,370.1,1.643868e+09,398.3,1.643889e+09,544.5,RUS,RUS,RUS,RUS,2248.0,2536,1242,2,4,8


In [ ]:
datafr_25814 = datafr

In [ ]:
path = 'C:\\Users\michele.lenato\Documents\MASTER-UNIMI\Lezioni\Python\progetto\definitivo_after_weights.xlsx'  # Inserire il path in cui è stato scaricato il file input del repo github
input_3 = pd.read_excel(path) 
del input_3['Unnamed: 0']
input_3

SECONDA ITERAZIONE. Aggiunta dei nodi A mancanti, reiterando i passi precedenti

In [ ]:
mancanti = [x for x in XX['id'].values.tolist() if x not in input_3['nodoA'].values.tolist() ]
len(mancanti)

In [17]:
#solo per i mancanti
n_lat = 5000
n_lon = 200

diz_lat_test = {}
for id in mancanti:
   l = list(GDM_lat.loc[GDM_lat[GDM_lat['id']==id].index[0]:]['id'][:n_lat]) +list(GDM_lat.loc[:GDM_lat[GDM_lat['id']==id].index[0]]['id'][-n_lat:])  
   diz_lat_test[GDM_lat[GDM_lat['id']==id]['id'].values[0]] = l

diz_lnt_test = {}
for id in mancanti:
    diz_lnt_test[X[X['id']==id]['id'].values[0]] = list(X.loc[X[X['id']==id].index[0]:]['id'][:n_lon])


cluster_ll_test = {}
saltati_test = []
for k, v in diz_lnt_test.items():
  try:
   cluster_ll_test[k] = [x for x in v if x in diz_lat_test[k]]
  except KeyError:
    saltati_test.append(k)
    continue

In [18]:
#solo per i mancanti
cl = pd.DataFrame(cluster_ll_test.items(), columns=['nodo_a', 'nodi_z'])
zz = XX.merge(cl, left_on='id', right_on='nodo_a') # inner join con il dataset iniziale per aggiungere coordinate, nomi, iso3 e popolazione

In [19]:
zz.shape

(755, 9)

In [20]:
#aggiunta coordinate solo mancanti
coord = pd.DataFrame()
for idx in (zz.index):
  temp = pd.DataFrame()
  lista_ll = []
  for x in zz.loc[idx][8:].values.tolist()[0]:
    try:
      if x is None or int(x)==0:
        pass
      elif int(x)!=0:
        lista_ll.append(XX[XX['id']==int(x)]['ll'].values[0]) 
    except ValueError:
      pass
  temp['id'] = [int(zz.loc[idx]['id'])]
  temp['llz'] = [lista_ll]
  coord = pd.concat([coord, temp])

fin = pd.DataFrame()
fin = zz.merge(coord, left_on='id', right_on='id')

In [21]:
fin.shape

(755, 10)

In [22]:
def nodi_candidati_mancanti(fin):
  '''calcola il df "finale" con i nodi A-Z e distanze min, med, max dato il df temporaneo con i possibili candidati'''
  finale = pd.DataFrame()

  for j in range(0,fin.shape[0]): 

    temp = pd.DataFrame()
    nodes = [fin.loc[j,:]['id']] + nodi(fin.loc[j][8:].tolist()[0])[0] #somma primo nodo (A) della lista agli altri (Z) # decommentare se per i mancanti NB [0]
    #nodes = [fin.loc[j,:]['id']] + nodi(fin.loc[j][8:].tolist())[0]  #decommentare se per tutti i nodi 
    coord = [fin.loc[j,:]['ll']] + fin.loc[j]['llz']  #le loro coordinate
    #print(coord)
    cand = []
    cand_fin = []
    for_df = []
    for n, c in zip(nodes, coord):
      cand.append((int(n), c))
   
    for i in range(1, len(cand)):
      dist_a_z = round(geopy.distance.distance(cand[0][1], cand[i][1]).km, 1)
      if dist_a_z > 1.0:  #escludo eventuali nodi A-Z coincidenti
        if (cand[i][0], dist_a_z) not in cand_fin:  #VERIFICO SE NON CI SONO DOPPIONI NELLA LISTA CAND
          cand_fin.append((cand[i][0], dist_a_z))
    for_df = sorted(cand_fin, key=lambda x: x[1])[:3] #prendo i primi 3 Z dopo averli ordinati

    if len(for_df)==3:
      temp['nodoA'] = [cand[0][0]] # [fin.loc[j,:]['id']] # invece di [cand[0][0]]
      temp['nodoZmin'] = [for_df[0][0]]
      temp['dist_min'] = [for_df[0][1]]
      temp['nodoZmed'] = [for_df[1][0]]
      temp['dist_med'] = [for_df[1][1]]
      temp['nodoZmax'] = [for_df[2][0]]
      temp['dist_max'] = [for_df[2][1]]
      finale = pd.concat([finale, temp])
      #print(temp['nodoZmax'].values)
    elif len(for_df)==2:
      temp['nodoA'] = [cand[0][0]] #[fin.loc[j,:]['id']]  #[cand[0][0]]
      temp['nodoZmin'] = [for_df[0][0]]
      temp['dist_min'] = [for_df[0][1]]
      temp['nodoZmed'] = [for_df[1][0]]
      temp['dist_med'] = [for_df[1][1]]
      finale = pd.concat([finale, temp])
    elif len(for_df)==1:
      temp['nodoA'] = [cand[0][0]] #[fin.loc[j,:]['id']]
      temp['nodoZmin'] = [for_df[0][0]]
      temp['dist_min'] = [for_df[0][1]]
      finale = pd.concat([finale, temp])  

    elif len(for_df)==0:
      pass
 

  return finale

In [24]:
datafr = pd.DataFrame()
datafr = nodi_candidati_mancanti(fin)

In [25]:
#query su XX per aggiungere iso3 e popolazione

#isoA
fin7 = pd.DataFrame()
for n in datafr['nodoA'].values:
  tem = pd.DataFrame()
  try:
      tem['isoA'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoA'] = ['null']
  fin7 = pd.concat([fin7, tem])
  
#isoz1
fin1 = pd.DataFrame()
for n in datafr['nodoZmin'].values:
  tem = pd.DataFrame()
  try:
      tem['isoZ1'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoZ1'] = ['null']
  fin1 = pd.concat([fin1, tem])

#isoz2
fin2 = pd.DataFrame()
for n in datafr['nodoZmed'].values:
  tem = pd.DataFrame()
  try:
      tem['isoZ2'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoZ2'] = ['null']
  fin2 = pd.concat([fin2, tem])


#pop1
fin4 = pd.DataFrame()
for n in datafr['nodoZmin'].values:
  tem = pd.DataFrame()
  try:
      tem['popZ1'] = [XX[XX['id']==n]['population'].values[0]]
  except IndexError :
      tem['popZ1'] = ['null']
  fin4 = pd.concat([fin4, tem])

#pop2
fin5 = pd.DataFrame()
for n in datafr['nodoZmed'].values:
  tem = pd.DataFrame()
  try:
      tem['popZ2'] = [XX[XX['id']==n]['population'].values[0]]
  except IndexError :
      tem['popZ2'] = ['null']
  fin5 = pd.concat([fin5, tem])



#isoz3
fin3 = pd.DataFrame()
for n in datafr['nodoZmax'].values:
  tem = pd.DataFrame()
  try:
      tem['isoZ3'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoZ3'] = ['null']
  fin3 = pd.concat([fin3, tem])
#pop3
fin6 = pd.DataFrame()
for n in datafr['nodoZmax'].values:
  tem = pd.DataFrame()
  try:
      tem['popZ3'] = [XX[XX['id']==n]['population'].values[0]]
  except IndexError :
      tem['popZ3'] = ['null']
  fin6 = pd.concat([fin6, tem])

In [26]:
def flatten(L):
    for l in L:
        if isinstance(l, list):
            yield from flatten(l)
        else:
            yield l

isoA = list(flatten(list(flatten(fin7.values.tolist()))))
popZ3 = list(flatten(list(flatten(fin6.values.tolist()))))
popZ2 = list(flatten(list(flatten(fin5.values.tolist()))))
popZ1 = list(flatten(list(flatten(fin4.values.tolist()))))
isoZ3 = list(flatten(list(flatten(fin3.values.tolist()))))
isoZ2 = list(flatten(list(flatten(fin2.values.tolist()))))
isoZ1 = list(flatten(list(flatten(fin1.values.tolist()))))

datafr['isoA'] = isoA
datafr['isoZ1'] = isoZ1
datafr['isoZ2'] = isoZ2
datafr['isoZ3'] = isoZ3
datafr['popZ1'] = popZ1
datafr['popZ2'] = popZ2
datafr['popZ3'] = popZ3

In [27]:
datafr['weight_nodo_vicino'] = [aggiorna_peso_nodo_vicino_b( isoA, isoZ1,popZ1) for isoA, isoZ1,popZ1 in zip( datafr.isoA.values, datafr.isoZ1.values, datafr.popZ1.values)]
datafr['weight_nodo_medio'] = [aggiorna_peso_nodo_medio_b(isoA,  isoZ2, popZ2) for  isoA, isoZ2,popZ2 in zip(  datafr.isoA.values, datafr.isoZ2.values, datafr.popZ2.values)]
datafr['weight_nodo_massimo'] = [aggiorna_peso_nodo_massimo_b( isoA, isoZ3,popZ3) for isoA, isoZ3,popZ3 in zip( datafr.isoA.values, datafr.isoZ3.values, datafr.popZ3.values)]

In [29]:
datafr[:1]

,nodoA,nodoZmin,dist_min,nodoZmed,dist_med,nodoZmax,dist_max,isoA,isoZ1,isoZ2,isoZ3,popZ1,popZ2,popZ3,weight_nodo_vicino,weight_nodo_medio,weight_nodo_massimo
0,1554847194,1776737461,2533.5,1.776830e+09,2815.8,1.184218e+09,2961.3,NZL,TON,TON,COK,24571.0,7391,5445,4,6,10


In [28]:
datafr.shape

(749, 17)

In [30]:
datafr_749 = datafr

In [31]:
#unione dei due df
datafr_tutti = pd.concat([datafr_25814, datafr_749])
datafr_tutti_finale = datafr_tutti.reset_index()
del datafr_tutti_finale['index']
datafr_tutti_finale

,nodoA,nodoZmin,dist_min,nodoZmed,dist_med,nodoZmax,dist_max,isoA,isoZ1,isoZ2,isoZ3,popZ1,popZ2,popZ3,weight_nodo_vicino,weight_nodo_medio,weight_nodo_massimo
0,1643877771,1643466779,517.2,1.643693e+09,544.0,1.643387e+09,666.4,RUS,RUS,RUS,RUS,2248.0,297,776,2,4,8
1,1643466779,1643693470,333.4,1.643868e+09,348.5,1.643889e+09,379.9,RUS,RUS,RUS,RUS,297.0,2536,1242,2,4,8
2,1876731744,1882504626,602.7,1.882879e+09,629.2,1.882940e+09,632.6,WLF,WSM,WSM,WSM,0.0,0,0,4,6,10
3,1776737461,1776829745,303.5,1.184218e+09,1602.6,1.258907e+09,2720.7,TON,TON,COK,PYF,7391.0,5445,131695,2,6,10
4,1776829745,1882940042,568.7,1.882048e+09,570.5,1.882030e+09,572.3,TON,WSM,WSM,WSM,0.0,0,0,4,6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26558,1643689971,1643981807,196.1,1.643010e+09,220.5,1.643675e+09,295.7,RUS,RUS,RUS,RUS,10.0,67386,21973,2,4,8
26559,1643129247,1643863467,392.0,1.643715e+09,431.6,1.643585e+09,680.4,RUS,RUS,RUS,RUS,1089.0,4939,3671,2,4,8
26560,1643715088,1643660854,430.4,1.643630e+09,499.7,1.643406e+09,621.0,RUS,RUS,RUS,RUS,10.0,400,10,2,4,8
26561,1584000000,1296060597,664.7,1.296153e+09,664.7,1.798251e+09,1935.8,MHL,KIR,KIR,TUV,5105.0,28802,6025,4,6,10


TERZA ITERAZIONE: stessi passi della seconda iterazione applicati all'ulyimo dataframe ottenuto: "datafr_tutti_finale"

In [43]:
datafr_tutti_finale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26563 entries, 0 to 26562
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   nodoA                26563 non-null  int64  
 1   nodoZmin             26563 non-null  int64  
 2   dist_min             26563 non-null  float64
 3   nodoZmed             25586 non-null  float64
 4   dist_med             25586 non-null  float64
 5   nodoZmax             24558 non-null  float64
 6   dist_max             24558 non-null  float64
 7   isoA                 26563 non-null  object 
 8   isoZ1                26563 non-null  object 
 9   isoZ2                26563 non-null  object 
 10  isoZ3                26563 non-null  object 
 11  popZ1                26563 non-null  float64
 12  popZ2                26563 non-null  object 
 13  popZ3                26563 non-null  object 
 14  weight_nodo_vicino   26563 non-null  int64  
 15  weight_nodo_medio    26563 non-null 

In [86]:
mancanti_a_me = input_3[input_3['nodoZmed'].isnull()]['nodoA'].values.tolist()
mancanti_a_ma = input_3[input_3['nodoZmax'].isnull()]['nodoA'].values.tolist()
mancanti = list(set(mancanti_a_me + mancanti_a_ma))

len(mancanti)

4156

In [87]:
#solo per i mancanti
n_lat = 6000
n_lon = 250

diz_lat_test = {}
for id in mancanti:
   l = list(GDM_lat.loc[GDM_lat[GDM_lat['id']==id].index[0]:]['id'][:n_lat]) +list(GDM_lat.loc[:GDM_lat[GDM_lat['id']==id].index[0]]['id'][-n_lat:])  
   diz_lat_test[GDM_lat[GDM_lat['id']==id]['id'].values[0]] = l

diz_lnt_test = {}
for id in mancanti:
    diz_lnt_test[X[X['id']==id]['id'].values[0]] = list(X.loc[X[X['id']==id].index[0]:]['id'][:n_lon])


cluster_ll_test = {}
saltati_test = []
for k, v in diz_lnt_test.items():
  try:
   cluster_ll_test[k] = [x for x in v if x in diz_lat_test[k]]
  except KeyError:
    saltati_test.append(k)
    continue

In [88]:
#solo per i mancanti
cl = pd.DataFrame(cluster_ll_test.items(), columns=['nodo_a', 'nodi_z'])
zz = XX.merge(cl, left_on='id', right_on='nodo_a')

#aggiunta coordinate solo mancanti
coord = pd.DataFrame()
for idx in (zz.index):
  temp = pd.DataFrame()
  lista_ll = []
  for x in zz.loc[idx][8:].values.tolist()[0]:
    try:
      if x is None or int(x)==0:
        pass
      elif int(x)!=0:
        lista_ll.append(XX[XX['id']==int(x)]['ll'].values[0]) 
    except ValueError:
      pass
  temp['id'] = [int(zz.loc[idx]['id'])]
  temp['llz'] = [lista_ll]
  coord = pd.concat([coord, temp])

fin = pd.DataFrame()
fin = zz.merge(coord, left_on='id', right_on='id')



In [89]:
datafr = pd.DataFrame()
datafr = nodi_candidati_mancanti(fin)

#query su XX per aggiungere iso3 e popolazione

#isoA
fin7 = pd.DataFrame()
for n in datafr['nodoA'].values:
  tem = pd.DataFrame()
  try:
      tem['isoA'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoA'] = ['null']
  fin7 = pd.concat([fin7, tem])
  
#isoz1
fin1 = pd.DataFrame()
for n in datafr['nodoZmin'].values:
  tem = pd.DataFrame()
  try:
      tem['isoZ1'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoZ1'] = ['null']
  fin1 = pd.concat([fin1, tem])

#isoz2
fin2 = pd.DataFrame()
for n in datafr['nodoZmed'].values:
  tem = pd.DataFrame()
  try:
      tem['isoZ2'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoZ2'] = ['null']
  fin2 = pd.concat([fin2, tem])


#pop1
fin4 = pd.DataFrame()
for n in datafr['nodoZmin'].values:
  tem = pd.DataFrame()
  try:
      tem['popZ1'] = [XX[XX['id']==n]['population'].values[0]]
  except IndexError :
      tem['popZ1'] = ['null']
  fin4 = pd.concat([fin4, tem])

#pop2
fin5 = pd.DataFrame()
for n in datafr['nodoZmed'].values:
  tem = pd.DataFrame()
  try:
      tem['popZ2'] = [XX[XX['id']==n]['population'].values[0]]
  except IndexError :
      tem['popZ2'] = ['null']
  fin5 = pd.concat([fin5, tem])



#isoz3
fin3 = pd.DataFrame()
for n in datafr['nodoZmax'].values:
  tem = pd.DataFrame()
  try:
      tem['isoZ3'] = [XX[XX['id']==n]['iso3'].values[0]]
  except IndexError :
      tem['isoZ3'] = ['null']
  fin3 = pd.concat([fin3, tem])
#pop3
fin6 = pd.DataFrame()
for n in datafr['nodoZmax'].values:
  tem = pd.DataFrame()
  try:
      tem['popZ3'] = [XX[XX['id']==n]['population'].values[0]]
  except IndexError :
      tem['popZ3'] = ['null']
  fin6 = pd.concat([fin6, tem])

isoA = list(flatten(list(flatten(fin7.values.tolist()))))
popZ3 = list(flatten(list(flatten(fin6.values.tolist()))))
popZ2 = list(flatten(list(flatten(fin5.values.tolist()))))
popZ1 = list(flatten(list(flatten(fin4.values.tolist()))))
isoZ3 = list(flatten(list(flatten(fin3.values.tolist()))))
isoZ2 = list(flatten(list(flatten(fin2.values.tolist()))))
isoZ1 = list(flatten(list(flatten(fin1.values.tolist()))))

datafr['isoA'] = isoA
datafr['isoZ1'] = isoZ1
datafr['isoZ2'] = isoZ2
datafr['isoZ3'] = isoZ3
datafr['popZ1'] = popZ1
datafr['popZ2'] = popZ2
datafr['popZ3'] = popZ3

datafr['weight_nodo_vicino'] = [aggiorna_peso_nodo_vicino_b( isoA, isoZ1,popZ1) for isoA, isoZ1,popZ1 in zip( datafr.isoA.values, datafr.isoZ1.values, datafr.popZ1.values)]
datafr['weight_nodo_medio'] = [aggiorna_peso_nodo_medio_b(isoA,  isoZ2, popZ2) for  isoA, isoZ2,popZ2 in zip(  datafr.isoA.values, datafr.isoZ2.values, datafr.popZ2.values)]
datafr['weight_nodo_massimo'] = [aggiorna_peso_nodo_massimo_b( isoA, isoZ3,popZ3) for isoA, isoZ3,popZ3 in zip( datafr.isoA.values, datafr.isoZ3.values, datafr.popZ3.values)]

In [90]:
datafr_3_iterazione = datafr

In [105]:
datafr_3_iterazione

,nodoA,nodoZmin,dist_min,nodoZmed,dist_med,nodoZmax,dist_max,isoA,isoZ1,isoZ2,isoZ3,popZ1,popZ2,popZ3,weight_nodo_vicino,weight_nodo_medio,weight_nodo_massimo
0,1643693470,1643387315,125.8,1.643889e+09,156.5,1.840075e+09,1104.1,RUS,RUS,RUS,USA,776.0,1242,5903,2,4,10
0,1882468874,1016976740,96.9,1.184218e+09,1484.4,1.258907e+09,2386.0,WSM,ASM,COK,PYF,12576.0,5445,131695,4,6,10
0,1643889063,1643387315,84.4,1.840075e+09,1074.4,1.840023e+09,1082.1,RUS,RUS,USA,USA,776.0,5903,13709,2,6,10
0,1016976740,1184217570,1389.6,1.258907e+09,2290.8,1.840029e+09,4084.0,ASM,COK,PYF,USA,5445.0,131695,9136,4,6,10
0,1840137122,1840023193,68.0,1.840030e+09,84.7,1.840020e+09,3748.9,USA,USA,USA,USA,46284.0,11479,38546,2,4,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1643597133,1643631484,387.1,1.643878e+09,403.9,1.643467e+09,577.1,RUS,RUS,RUS,RUS,100.0,10,2248,2,4,8
0,1584000000,1296060597,664.7,1.296153e+09,664.7,1.798251e+09,1935.8,MHL,KIR,KIR,TUV,5105.0,28802,6025,4,6,10
0,1296060597,1296152641,2.0,1.798251e+09,1289.3,1.876732e+09,1984.2,KIR,KIR,TUV,WLF,28802.0,6025,0,2,6,10
0,1554253729,1554359840,119.1,1.554847e+09,858.1,1.776737e+09,2011.1,NZL,NZL,NZL,TON,35500.0,300,24571,2,4,10


In [107]:
#unione dei df di 2 e 3 iterazione
datafr_3_b = pd.concat([input_3, datafr_3_iterazione])
datafr_3_3_b = datafr_3_b.reset_index()
del datafr_3_3_b['index']
datafr_3_3_b  

,nodoA,nodoZmin,dist_min,nodoZmed,dist_med,nodoZmax,dist_max,isoA,isoZ1,isoZ2,isoZ3,popZ1,popZ2,popZ3,weight_nodo_vicino,weight_nodo_medio,weight_nodo_massimo
0,1643877771,1643466779,517.2,1.643693e+09,544.0,1.643387e+09,666.4,RUS,RUS,RUS,RUS,2248.0,297,776,2,4,8.0
1,1643466779,1643693470,333.4,1.643868e+09,348.5,1.643889e+09,379.9,RUS,RUS,RUS,RUS,297.0,2536,1242,2,4,8.0
2,1876731744,1882504626,602.7,1.882879e+09,629.2,1.882940e+09,632.6,WLF,WSM,WSM,WSM,0.0,0,0,4,6,10.0
3,1776737461,1776829745,303.5,1.882940e+09,871.0,1.882048e+09,872.8,TON,TON,WSM,WSM,7391.0,0,0,2,6,10.0
4,1776829745,1882940042,568.7,1.882048e+09,570.5,1.882030e+09,572.3,TON,WSM,WSM,WSM,0.0,0,0,4,6,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30718,1643597133,1643631484,387.1,1.643878e+09,403.9,1.643467e+09,577.1,RUS,RUS,RUS,RUS,100.0,10,2248,2,4,8.0
30719,1584000000,1296060597,664.7,1.296153e+09,664.7,1.798251e+09,1935.8,MHL,KIR,KIR,TUV,5105.0,28802,6025,4,6,10.0
30720,1296060597,1296152641,2.0,1.798251e+09,1289.3,1.876732e+09,1984.2,KIR,KIR,TUV,WLF,28802.0,6025,0,2,6,10.0
30721,1554253729,1554359840,119.1,1.554847e+09,858.1,1.776737e+09,2011.1,NZL,NZL,NZL,TON,35500.0,300,24571,2,4,10.0


Dataframe con le città source e le 3 città più vicine

In [108]:
vicini= datafr_3_3_b[['nodoA', 'nodoZmin', 'weight_nodo_vicino']]
medi = datafr_3_3_b[['nodoA', 'nodoZmed', 'weight_nodo_medio']]
massimi = datafr_3_3_b[['nodoA', 'nodoZmax', 'weight_nodo_massimo']]
dv = {'nodoA': 'A', 'nodoZmin': 'Z', 'weight_nodo_vicino': 'W'}
dme = {'nodoA': 'A', 'nodoZmed': 'Z', 'weight_nodo_medio': 'W'}
dma = {'nodoA': 'A', 'nodoZmax': 'Z', 'weight_nodo_massimo': 'W'}
vicini.rename(columns = dv, inplace = True)
medi.rename(columns = dme, inplace = True)
massimi.rename(columns = dma, inplace = True)

C:\Users\michele.lenato\Anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [109]:
# esclusione nodi Z inesistenti
v = vicini[vicini['Z']!=0]
me = medi[medi['Z']!=0]
ma = massimi[massimi['Z']!=0]
v_b = v.dropna()
me_b = me.dropna()
ma_b= ma.dropna()

In [110]:
l = [v_b,me_b,ma_b]
tot = pd.DataFrame()
tot = pd.concat(l)
t = tot.reset_index()
t = t.astype(int)
t_fin = t.drop_duplicates()
t_fin2 = t_fin.reset_index()
del t_fin2['index']
del t_fin2['level_0']

In [111]:
t_fin2

,A,Z,W
0,1643877771,1643466779,2
1,1643466779,1643693470,2
2,1876731744,1882504626,4
3,1776737461,1776829745,2
4,1776829745,1882940042,4
...,...,...,...
85911,1643597133,1643466779,8
85912,1584000000,1798251391,10
85913,1296060597,1876731744,10
85914,1554253729,1776737461,10


In [ ]:
t_fin3 = t_fin2.drop_duplicates()
t_fin4 = t_fin3.reset_index()
del t_fin4['index']

Prendo i 3 nodi più vicini perchè ve ne sono più di 3

In [174]:
lista_nodi = XX['id'].values.tolist()
df_input_nx_3 = pd.DataFrame()
for n in lista_nodi:
    tem = pd.DataFrame()
    tem = t_fin4[t_fin4['A'] == n]
    tem2 = tem.sort_values(by='W', ascending=True)
    tem3 = tem2[:3]
    df_input_nx_3 = pd.concat([df_input_nx_3, tem3])


In [175]:
df_input_nx_3

,A,Z,W
0,1643877771,1643466779,2
27995,1643877771,1643693470,4
55790,1643877771,1643387315,8
1,1643466779,1643693470,2
27996,1643466779,1643868459,4
...,...,...,...
50534,1643017874,1643868459,4
76292,1643017874,1643889063,8
24607,1242740801,1876731744,4
50535,1242740801,1776829745,6


In [178]:
df_input_nx_3.to_excel('C:\\Users\michele.lenato\Documents\MASTER-UNIMI\Lezioni\Python\progetto\input.xlsx')

In [185]:
df_input_nx_3.to_csv('C:\\Users\michele.lenato\Documents\MASTER-UNIMI\Lezioni\Python\progetto\input.csv')

Test su connessione e cicli nel grafo

Grafo presentato

In [180]:
path = 'C:\\Users\michele.lenato\Documents\MASTER-UNIMI\Lezioni\Python\progetto\input.xlsx'  # Inserire il path in cui è stato scaricato il file input del repo github
input = pd.read_excel(path) 
del input['Unnamed: 0']

In [181]:
a_non_trovati = []
nodi_a_scansionati = []

def tree(input, nodi_iniziali):
  '''
  Input: df di input a networkx.
  La condizione di arresto è:
  1- se trova Londra a partire da Holborn 1826657058 (o  qualsiasi altro nodo)
  2- non trova più nodi (presenza di cicli o grafo sconnesso)
  '''
  global nodi_a_scansionati 
  global a_non_trovati
  lista_z_tot = [] #lista di nodi z per ogni ricorsione

  if 1826645935 in nodi_iniziali: 
    print('fine perchè ha trovato Londra')  
    return 
  if len(nodi_iniziali) == 0:
    print('fine perchè non trova più nodi')
    return nodi_iniziali, lista_z_tot, a_non_trovati
  for nodo_a in set(nodi_iniziali)  -  set(flatten(nodi_a_scansionati)) :

    lista_z = [] #lista temporanea per stoccare la lista di nodi z di ciascun nodo a
    if nodo_a not in input['A'].tolist():
      a_non_trovati.append(nodo_a)
      print('FINE')

    if nodo_a in input['A'].tolist(): #and nodo_a not in nodi_a_scansionati:
      nodi_a_scansionati.append(nodo_a)

      lista_z = input[input['A']==nodo_a]['Z'].tolist()

      lista_z_tot.append(lista_z)

      print('nodi scansionati', len(set(list(flatten(nodi_a_scansionati)))))

      if len(set(list(flatten(nodi_a_scansionati))))==26569:  # fine scansione
        print('fine per esaurimento di 26569 nodi')
        return nodi_iniziali, lista_z_tot, a_non_trovati
  
  
  return tree(input, list(set(flatten(lista_z_tot)) ))

Grafo a 3 neighbor

In [182]:
# resettare le liste iniziali
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

tree(input, [1826657058]) 

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 01:14:30
nodi scansionati 1
nodi scansionati 2
nodi scansionati 3
nodi scansionati 4
nodi scansionati 5
nodi scansionati 6
nodi scansionati 7
nodi scansionati 8
nodi scansionati 9
nodi scansionati 10
nodi scansionati 11
nodi scansionati 12
nodi scansionati 13
nodi scansionati 14
nodi scansionati 15
nodi scansionati 16
nodi scansionati 17
nodi scansionati 18
nodi scansionati 19
nodi scansionati 20
nodi scansionati 21
nodi scansionati 22
nodi scansionati 23
nodi scansionati 24
nodi scansionati 25
nodi scansionati 26
nodi scansionati 27
nodi scansionati 28
nodi scansionati 29
nodi scansionati 30
nodi scansionati 31
nodi scansionati 32
nodi scansionati 33
nodi scansionati 34
nodi scansionati 35
nodi scansionati 36
nodi scansionati 37
nodi scansionati 38
nodi scansionati 39
nodi scansionati 40
nodi scansionati 41
nodi scansionati 42
nodi scansionati 43
nodi scansionati 44
nodi scansionati 45
nodi scansionati 46
nodi scansionati 47
nodi scansionati 48
nodi scansionati 49
nodi 

nodi scansionati 412
nodi scansionati 413
nodi scansionati 414
nodi scansionati 415
nodi scansionati 416
nodi scansionati 417
nodi scansionati 418
nodi scansionati 419
nodi scansionati 420
nodi scansionati 421
nodi scansionati 422
nodi scansionati 423
nodi scansionati 424
nodi scansionati 425
nodi scansionati 426
nodi scansionati 427
nodi scansionati 428
nodi scansionati 429
nodi scansionati 430
nodi scansionati 431
nodi scansionati 432
nodi scansionati 433
nodi scansionati 434
nodi scansionati 435
nodi scansionati 436
nodi scansionati 437
nodi scansionati 438
nodi scansionati 439
nodi scansionati 440
nodi scansionati 441
nodi scansionati 442
nodi scansionati 443
nodi scansionati 444
nodi scansionati 445
nodi scansionati 446
nodi scansionati 447
nodi scansionati 448
nodi scansionati 449
nodi scansionati 450
nodi scansionati 451
nodi scansionati 452
nodi scansionati 453
nodi scansionati 454
nodi scansionati 455
nodi scansionati 456
nodi scansionati 457
nodi scansionati 458
nodi scansion

nodi scansionati 803
nodi scansionati 804
nodi scansionati 805
nodi scansionati 806
nodi scansionati 807
nodi scansionati 808
nodi scansionati 809
nodi scansionati 810
nodi scansionati 811
nodi scansionati 812
nodi scansionati 813
nodi scansionati 814
nodi scansionati 815
nodi scansionati 816
nodi scansionati 817
nodi scansionati 818
nodi scansionati 819
nodi scansionati 820
nodi scansionati 821
nodi scansionati 822
nodi scansionati 823
nodi scansionati 824
nodi scansionati 825
nodi scansionati 826
nodi scansionati 827
nodi scansionati 828
nodi scansionati 829
nodi scansionati 830
nodi scansionati 831
nodi scansionati 832
nodi scansionati 833
nodi scansionati 834
nodi scansionati 835
nodi scansionati 836
nodi scansionati 837
nodi scansionati 838
nodi scansionati 839
nodi scansionati 840
nodi scansionati 841
nodi scansionati 842
nodi scansionati 843
nodi scansionati 844
nodi scansionati 845
nodi scansionati 846
nodi scansionati 847
nodi scansionati 848
nodi scansionati 849
nodi scansion

nodi scansionati 1191
nodi scansionati 1192
nodi scansionati 1193
nodi scansionati 1194
nodi scansionati 1195
nodi scansionati 1196
nodi scansionati 1197
nodi scansionati 1198
nodi scansionati 1199
nodi scansionati 1200
nodi scansionati 1201
nodi scansionati 1202
nodi scansionati 1203
nodi scansionati 1204
nodi scansionati 1205
nodi scansionati 1206
nodi scansionati 1207
nodi scansionati 1208
nodi scansionati 1209
nodi scansionati 1210
nodi scansionati 1211
nodi scansionati 1212
nodi scansionati 1213
nodi scansionati 1214
nodi scansionati 1215
nodi scansionati 1216
nodi scansionati 1217
nodi scansionati 1218
nodi scansionati 1219
nodi scansionati 1220
nodi scansionati 1221
nodi scansionati 1222
nodi scansionati 1223
nodi scansionati 1224
nodi scansionati 1225
nodi scansionati 1226
nodi scansionati 1227
nodi scansionati 1228
nodi scansionati 1229
nodi scansionati 1230
nodi scansionati 1231
nodi scansionati 1232
nodi scansionati 1233
nodi scansionati 1234
nodi scansionati 1235
nodi scans

nodi scansionati 1568
nodi scansionati 1569
nodi scansionati 1570
nodi scansionati 1571
nodi scansionati 1572
nodi scansionati 1573
nodi scansionati 1574
nodi scansionati 1575
nodi scansionati 1576
nodi scansionati 1577
nodi scansionati 1578
nodi scansionati 1579
nodi scansionati 1580
nodi scansionati 1581
nodi scansionati 1582
nodi scansionati 1583
nodi scansionati 1584
nodi scansionati 1585
nodi scansionati 1586
nodi scansionati 1587
nodi scansionati 1588
nodi scansionati 1589
nodi scansionati 1590
nodi scansionati 1591
nodi scansionati 1592
nodi scansionati 1593
nodi scansionati 1594
nodi scansionati 1595
nodi scansionati 1596
nodi scansionati 1597
nodi scansionati 1598
nodi scansionati 1599
nodi scansionati 1600
nodi scansionati 1601
nodi scansionati 1602
nodi scansionati 1603
nodi scansionati 1604
nodi scansionati 1605
nodi scansionati 1606
nodi scansionati 1607
nodi scansionati 1608
nodi scansionati 1609
nodi scansionati 1610
nodi scansionati 1611
nodi scansionati 1612
nodi scans

...

nodi scansionati 17852
nodi scansionati 17853
nodi scansionati 17854
nodi scansionati 17855
nodi scansionati 17856
nodi scansionati 17857
nodi scansionati 17858
nodi scansionati 17859
nodi scansionati 17860
nodi scansionati 17861
nodi scansionati 17862
nodi scansionati 17863
nodi scansionati 17864
nodi scansionati 17865
nodi scansionati 17866
nodi scansionati 17867
nodi scansionati 17868
nodi scansionati 17869
nodi scansionati 17870
nodi scansionati 17871
nodi scansionati 17872
nodi scansionati 17873
nodi scansionati 17874
nodi scansionati 17875
nodi scansionati 17876
nodi scansionati 17877
nodi scansionati 17878
nodi scansionati 17879
nodi scansionati 17880
nodi scansionati 17881
nodi scansionati 17882
nodi scansionati 17883
nodi scansionati 17884
nodi scansionati 17885
nodi scansionati 17886
nodi scansionati 17887
nodi scansionati 17888
nodi scansionati 17889
nodi scansionati 17890
nodi scansionati 17891
nodi scansionati 17892
nodi scansionati 17893
nodi scansionati 17894
nodi scansi

nodi scansionati 18215
nodi scansionati 18216
nodi scansionati 18217
nodi scansionati 18218
nodi scansionati 18219
nodi scansionati 18220
nodi scansionati 18221
nodi scansionati 18222
nodi scansionati 18223
nodi scansionati 18224
nodi scansionati 18225
nodi scansionati 18226
nodi scansionati 18227
nodi scansionati 18228
nodi scansionati 18229
nodi scansionati 18230
nodi scansionati 18231
nodi scansionati 18232
nodi scansionati 18233
nodi scansionati 18234
nodi scansionati 18235
nodi scansionati 18236
nodi scansionati 18237
nodi scansionati 18238
nodi scansionati 18239
nodi scansionati 18240
nodi scansionati 18241
nodi scansionati 18242
nodi scansionati 18243
nodi scansionati 18244
nodi scansionati 18245
nodi scansionati 18246
nodi scansionati 18247
nodi scansionati 18248
nodi scansionati 18249
nodi scansionati 18250
nodi scansionati 18251
nodi scansionati 18252
nodi scansionati 18253
nodi scansionati 18254
nodi scansionati 18255
nodi scansionati 18256
nodi scansionati 18257
nodi scansi

nodi scansionati 18574
nodi scansionati 18575
nodi scansionati 18576
nodi scansionati 18577
nodi scansionati 18578
nodi scansionati 18579
nodi scansionati 18580
nodi scansionati 18581
nodi scansionati 18582
nodi scansionati 18583
nodi scansionati 18584
nodi scansionati 18585
nodi scansionati 18586
nodi scansionati 18587
nodi scansionati 18588
nodi scansionati 18589
nodi scansionati 18590
nodi scansionati 18591
nodi scansionati 18592
nodi scansionati 18593
nodi scansionati 18594
nodi scansionati 18595
nodi scansionati 18596
nodi scansionati 18597
nodi scansionati 18598
nodi scansionati 18599
nodi scansionati 18600
nodi scansionati 18601
nodi scansionati 18602
nodi scansionati 18603
nodi scansionati 18604
nodi scansionati 18605
nodi scansionati 18606
nodi scansionati 18607
nodi scansionati 18608
nodi scansionati 18609
nodi scansionati 18610
nodi scansionati 18611
nodi scansionati 18612
nodi scansionati 18613
nodi scansionati 18614
nodi scansionati 18615
nodi scansionati 18616
nodi scansi

nodi scansionati 19291
nodi scansionati 19292
nodi scansionati 19293
nodi scansionati 19294
nodi scansionati 19295
nodi scansionati 19296
nodi scansionati 19297
nodi scansionati 19298
nodi scansionati 19299
nodi scansionati 19300
nodi scansionati 19301
nodi scansionati 19302
nodi scansionati 19303
nodi scansionati 19304
nodi scansionati 19305
nodi scansionati 19306
nodi scansionati 19307
nodi scansionati 19308
nodi scansionati 19309
nodi scansionati 19310
nodi scansionati 19311
nodi scansionati 19312
nodi scansionati 19313
nodi scansionati 19314
nodi scansionati 19315
nodi scansionati 19316
nodi scansionati 19317
nodi scansionati 19318
nodi scansionati 19319
nodi scansionati 19320
nodi scansionati 19321
nodi scansionati 19322
nodi scansionati 19323
nodi scansionati 19324
nodi scansionati 19325
nodi scansionati 19326
nodi scansionati 19327
nodi scansionati 19328
nodi scansionati 19329
nodi scansionati 19330
nodi scansionati 19331
nodi scansionati 19332
nodi scansionati 19333
nodi scansi

nodi scansionati 19656
nodi scansionati 19657
nodi scansionati 19658
nodi scansionati 19659
nodi scansionati 19660
nodi scansionati 19661
nodi scansionati 19662
nodi scansionati 19663
nodi scansionati 19664
nodi scansionati 19665
nodi scansionati 19666
nodi scansionati 19667
nodi scansionati 19668
nodi scansionati 19669
nodi scansionati 19670
nodi scansionati 19671
nodi scansionati 19672
nodi scansionati 19673
nodi scansionati 19674
nodi scansionati 19675
nodi scansionati 19676
nodi scansionati 19677
nodi scansionati 19678
nodi scansionati 19679
nodi scansionati 19680
nodi scansionati 19681
nodi scansionati 19682
nodi scansionati 19683
nodi scansionati 19684
nodi scansionati 19685
nodi scansionati 19686
nodi scansionati 19687
nodi scansionati 19688
nodi scansionati 19689
nodi scansionati 19690
nodi scansionati 19691
nodi scansionati 19692
nodi scansionati 19693
nodi scansionati 19694
nodi scansionati 19695
nodi scansionati 19696
nodi scansionati 19697
nodi scansionati 19698
nodi scansi

In [73]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

tree(input, [1826657058]) 

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 22:04:12
nodi scansionati 1
nodi scansionati 2
nodi scansionati 3
nodi scansionati 4
nodi scansionati 5
nodi scansionati 6
nodi scansionati 7
nodi scansionati 8
nodi scansionati 9
nodi scansionati 10
nodi scansionati 11
nodi scansionati 12
nodi scansionati 13
nodi scansionati 14
nodi scansionati 15
nodi scansionati 16
nodi scansionati 17
nodi scansionati 18
nodi scansionati 19
nodi scansionati 20
nodi scansionati 21
nodi scansionati 22
nodi scansionati 23
nodi scansionati 24
nodi scansionati 25
nodi scansionati 26
nodi scansionati 27
nodi scansionati 28
nodi scansionati 29
nodi scansionati 30
nodi scansionati 31
nodi scansionati 32
nodi scansionati 33
nodi scansionati 34
nodi scansionati 35
nodi scansionati 36
nodi scansionati 37
nodi scansionati 38
nodi scansionati 39
nodi scansionati 40
nodi scansionati 41
nodi scansionati 42
nodi scansionati 43
nodi scansionati 44
nodi scansionati 45
nodi scansionati 46
nodi scansionati 47
nodi scansionati 48
nodi scansionati 49
nodi 

nodi scansionati 398
nodi scansionati 399
nodi scansionati 400
nodi scansionati 401
nodi scansionati 402
nodi scansionati 403
nodi scansionati 404
nodi scansionati 405
nodi scansionati 406
nodi scansionati 407
nodi scansionati 408
nodi scansionati 409
nodi scansionati 410
nodi scansionati 411
nodi scansionati 412
nodi scansionati 413
nodi scansionati 414
nodi scansionati 415
nodi scansionati 416
nodi scansionati 417
nodi scansionati 418
nodi scansionati 419
nodi scansionati 420
nodi scansionati 421
nodi scansionati 422
nodi scansionati 423
nodi scansionati 424
nodi scansionati 425
nodi scansionati 426
nodi scansionati 427
nodi scansionati 428
nodi scansionati 429
nodi scansionati 430
nodi scansionati 431
nodi scansionati 432
nodi scansionati 433
nodi scansionati 434
nodi scansionati 435
nodi scansionati 436
nodi scansionati 437
nodi scansionati 438
nodi scansionati 439
nodi scansionati 440
nodi scansionati 441
nodi scansionati 442
nodi scansionati 443
nodi scansionati 444
nodi scansion

nodi scansionati 798
nodi scansionati 799
nodi scansionati 800
nodi scansionati 801
nodi scansionati 802
nodi scansionati 803
nodi scansionati 804
nodi scansionati 805
nodi scansionati 806
nodi scansionati 807
nodi scansionati 808
nodi scansionati 809
nodi scansionati 810
nodi scansionati 811
nodi scansionati 812
nodi scansionati 813
nodi scansionati 814
nodi scansionati 815
nodi scansionati 816
nodi scansionati 817
nodi scansionati 818
nodi scansionati 819
nodi scansionati 820
nodi scansionati 821
nodi scansionati 822
nodi scansionati 823
nodi scansionati 824
nodi scansionati 825
nodi scansionati 826
nodi scansionati 827
nodi scansionati 828
nodi scansionati 829
nodi scansionati 830
nodi scansionati 831
nodi scansionati 832
nodi scansionati 833
nodi scansionati 834
nodi scansionati 835
nodi scansionati 836
nodi scansionati 837
nodi scansionati 838
nodi scansionati 839
nodi scansionati 840
nodi scansionati 841
nodi scansionati 842
nodi scansionati 843
nodi scansionati 844
nodi scansion

nodi scansionati 1180
nodi scansionati 1181
nodi scansionati 1182
nodi scansionati 1183
nodi scansionati 1184
nodi scansionati 1185
nodi scansionati 1186
nodi scansionati 1187
nodi scansionati 1188
nodi scansionati 1189
nodi scansionati 1190
nodi scansionati 1191
nodi scansionati 1192
nodi scansionati 1193
nodi scansionati 1194
nodi scansionati 1195
nodi scansionati 1196
nodi scansionati 1197
nodi scansionati 1198
nodi scansionati 1199
nodi scansionati 1200
nodi scansionati 1201
nodi scansionati 1202
nodi scansionati 1203
nodi scansionati 1204
nodi scansionati 1205
nodi scansionati 1206
nodi scansionati 1207
nodi scansionati 1208
nodi scansionati 1209
nodi scansionati 1210
nodi scansionati 1211
nodi scansionati 1212
nodi scansionati 1213
nodi scansionati 1214
nodi scansionati 1215
nodi scansionati 1216
nodi scansionati 1217
nodi scansionati 1218
nodi scansionati 1219
nodi scansionati 1220
nodi scansionati 1221
nodi scansionati 1222
nodi scansionati 1223
nodi scansionati 1224
nodi scans

nodi scansionati 1557
nodi scansionati 1558
nodi scansionati 1559
nodi scansionati 1560
nodi scansionati 1561
nodi scansionati 1562
nodi scansionati 1563
nodi scansionati 1564
nodi scansionati 1565
nodi scansionati 1566
nodi scansionati 1567
nodi scansionati 1568
nodi scansionati 1569
nodi scansionati 1570
nodi scansionati 1571
nodi scansionati 1572
nodi scansionati 1573
nodi scansionati 1574
nodi scansionati 1575
nodi scansionati 1576
nodi scansionati 1577
nodi scansionati 1578
nodi scansionati 1579
nodi scansionati 1580
nodi scansionati 1581
nodi scansionati 1582
nodi scansionati 1583
nodi scansionati 1584
nodi scansionati 1585
nodi scansionati 1586
nodi scansionati 1587
nodi scansionati 1588
nodi scansionati 1589
nodi scansionati 1590
nodi scansionati 1591
nodi scansionati 1592
nodi scansionati 1593
nodi scansionati 1594
nodi scansionati 1595
nodi scansionati 1596
nodi scansionati 1597
nodi scansionati 1598
nodi scansionati 1599
nodi scansionati 1600
nodi scansionati 1601
nodi scans

...

nodi scansionati 24253
nodi scansionati 24254
nodi scansionati 24255
nodi scansionati 24256
nodi scansionati 24257
nodi scansionati 24258
nodi scansionati 24259
nodi scansionati 24260
nodi scansionati 24261
nodi scansionati 24262
nodi scansionati 24263
nodi scansionati 24264
nodi scansionati 24265
nodi scansionati 24266
nodi scansionati 24267
nodi scansionati 24268
nodi scansionati 24269
nodi scansionati 24270
nodi scansionati 24271
nodi scansionati 24272
nodi scansionati 24273
nodi scansionati 24274
nodi scansionati 24275
nodi scansionati 24276
nodi scansionati 24277
nodi scansionati 24278
nodi scansionati 24279
nodi scansionati 24280
nodi scansionati 24281
nodi scansionati 24282
nodi scansionati 24283
nodi scansionati 24284
nodi scansionati 24285
nodi scansionati 24286
nodi scansionati 24287
nodi scansionati 24288
nodi scansionati 24289
nodi scansionati 24290
nodi scansionati 24291
nodi scansionati 24292
nodi scansionati 24293
nodi scansionati 24294
nodi scansionati 24295
nodi scansi

nodi scansionati 24611
nodi scansionati 24612
nodi scansionati 24613
nodi scansionati 24614
nodi scansionati 24615
nodi scansionati 24616
nodi scansionati 24617
nodi scansionati 24618
nodi scansionati 24619
nodi scansionati 24620
nodi scansionati 24621
nodi scansionati 24622
nodi scansionati 24623
nodi scansionati 24624
nodi scansionati 24625
nodi scansionati 24626
nodi scansionati 24627
nodi scansionati 24628
nodi scansionati 24629
nodi scansionati 24630
nodi scansionati 24631
nodi scansionati 24632
nodi scansionati 24633
nodi scansionati 24634
nodi scansionati 24635
nodi scansionati 24636
nodi scansionati 24637
nodi scansionati 24638
nodi scansionati 24639
nodi scansionati 24640
nodi scansionati 24641
nodi scansionati 24642
nodi scansionati 24643
nodi scansionati 24644
nodi scansionati 24645
nodi scansionati 24646
nodi scansionati 24647
nodi scansionati 24648
nodi scansionati 24649
nodi scansionati 24650
nodi scansionati 24651
nodi scansionati 24652
nodi scansionati 24653
nodi scansi

nodi scansionati 24971
nodi scansionati 24972
nodi scansionati 24973
nodi scansionati 24974
nodi scansionati 24975
nodi scansionati 24976
nodi scansionati 24977
nodi scansionati 24978
nodi scansionati 24979
nodi scansionati 24980
nodi scansionati 24981
nodi scansionati 24982
nodi scansionati 24983
nodi scansionati 24984
nodi scansionati 24985
nodi scansionati 24986
nodi scansionati 24987
nodi scansionati 24988
nodi scansionati 24989
nodi scansionati 24990
nodi scansionati 24991
nodi scansionati 24992
nodi scansionati 24993
nodi scansionati 24994
nodi scansionati 24995
nodi scansionati 24996
nodi scansionati 24997
nodi scansionati 24998
nodi scansionati 24999
nodi scansionati 25000
nodi scansionati 25001
nodi scansionati 25002
nodi scansionati 25003
nodi scansionati 25004
nodi scansionati 25005
nodi scansionati 25006
nodi scansionati 25007
nodi scansionati 25008
nodi scansionati 25009
nodi scansionati 25010
nodi scansionati 25011
nodi scansionati 25012
nodi scansionati 25013
nodi scansi

nodi scansionati 25328
nodi scansionati 25329
nodi scansionati 25330
nodi scansionati 25331
nodi scansionati 25332
nodi scansionati 25333
nodi scansionati 25334
nodi scansionati 25335
nodi scansionati 25336
nodi scansionati 25337
nodi scansionati 25338
nodi scansionati 25339
nodi scansionati 25340
nodi scansionati 25341
nodi scansionati 25342
nodi scansionati 25343
nodi scansionati 25344
nodi scansionati 25345
nodi scansionati 25346
nodi scansionati 25347
nodi scansionati 25348
nodi scansionati 25349
nodi scansionati 25350
nodi scansionati 25351
nodi scansionati 25352
nodi scansionati 25353
nodi scansionati 25354
nodi scansionati 25355
nodi scansionati 25356
nodi scansionati 25357
nodi scansionati 25358
nodi scansionati 25359
nodi scansionati 25360
nodi scansionati 25361
nodi scansionati 25362
nodi scansionati 25363
nodi scansionati 25364
nodi scansionati 25365
nodi scansionati 25366
nodi scansionati 25367
nodi scansionati 25368
nodi scansionati 25369
nodi scansionati 25370
nodi scansi

nodi scansionati 25685
nodi scansionati 25686
nodi scansionati 25687
nodi scansionati 25688
nodi scansionati 25689
nodi scansionati 25690
nodi scansionati 25691
nodi scansionati 25692
nodi scansionati 25693
nodi scansionati 25694
nodi scansionati 25695
nodi scansionati 25696
nodi scansionati 25697
nodi scansionati 25698
nodi scansionati 25699
nodi scansionati 25700
nodi scansionati 25701
nodi scansionati 25702
nodi scansionati 25703
nodi scansionati 25704
nodi scansionati 25705
nodi scansionati 25706
nodi scansionati 25707
nodi scansionati 25708
nodi scansionati 25709
nodi scansionati 25710
nodi scansionati 25711
nodi scansionati 25712
nodi scansionati 25713
nodi scansionati 25714
nodi scansionati 25715
nodi scansionati 25716
nodi scansionati 25717
nodi scansionati 25718
nodi scansionati 25719
nodi scansionati 25720
nodi scansionati 25721
nodi scansionati 25722
nodi scansionati 25723
nodi scansionati 25724
nodi scansionati 25725
nodi scansionati 25726
nodi scansionati 25727
nodi scansi